In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# 基本用法

## 对话上下文：ConversationBufferMemory

In [2]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

history = ConversationBufferMemory()
history.save_context({"input": "你好啊"}, {"output": "你也好啊"})

print(history.load_memory_variables({}))

history.save_context({"input": "你再好啊"}, {"output": "你又好啊"})

print(history.load_memory_variables({}))

{'history': 'Human: 你好啊\nAI: 你也好啊'}
{'history': 'Human: 你好啊\nAI: 你也好啊\nHuman: 你再好啊\nAI: 你又好啊'}


## 保留对话窗口：ConversationBufferWindowMemory

In [3]:
from langchain.memory import ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(k=1)
window.save_context({"input": "第一轮问"}, {"output": "第一轮答"})
window.save_context({"input": "第二轮问"}, {"output": "第二轮答"})
window.save_context({"input": "第三轮问"}, {"output": "第三轮答"})
print(window.load_memory_variables({}))

{'history': 'Human: 第三轮问\nAI: 第三轮答'}


## 自动对历史信息做摘要：ConversationSummaryMemory

In [6]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    # buffer="The conversation is between a customer and a sales."
    buffer="以中文表示"
)
memory.save_context(
    {"input": "你好"}, {"output": "你好，我是你的AI助手。我能为你回答有关langchain的各种问题。"})

print(memory.load_memory_variables({}))

{'history': '\n人类问AI对人工智能的看法。AI认为人工智能是一种积极的力量，因为它能帮助人类发挥他们的全部潜力。人类向AI打招呼，AI回应说它是人类的AI助手，可以回答关于langchain的各种问题。'}


## 更多类型

- ConversationTokenBufferMemory: 根据 Token 数限定 Memory 大小
  - https://python.langchain.com/docs/modules/memory/types/token_buffer
- VectorStoreRetrieverMemory: 将 Memory 存储在向量数据库中，根据用户输入检索回最相关的部分
  - https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory

# RunnableWithMessageHistory

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个擅长{ability}的助手，每次返回20个以内的字即可"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
llm = ChatOpenAI()

# 构建链
chain = prompt | llm

In [45]:
# 现在要运行链，需要三个输入参数
chain.input_schema()

PromptInput(ability=None, history=None, input=None)

## 携带对话历史的内存版本

In [31]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [71]:
# with_message_history 需要一个配置项 session_id
with_message_history.config_specs

[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)]

In [52]:
# 第 1 次
with_message_history.invoke(
    {"ability": "数学", "input": "三角函数是什么意思?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='三角函数是一类描述角度与三角形边长之间关系的数学函数。')

In [8]:
# 第 2 次
with_message_history.invoke(
    {"ability": "数学", "input": "可以用小学三年级小朋友能听懂的话解释吗？"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='三角函数就是一种特殊的数学工具，可以帮助我们理解角度与三角形之间的关系，就像解开一个有趣的数学谜题一样。')

In [9]:
# 此时 store 中多了两条消息历史
store

{'abc123': ChatMessageHistory(messages=[HumanMessage(content='三角函数是什么意思?'), AIMessage(content='三角函数是一类数学函数，用于描述角度和三角形之间的关系。常见的三角函数包括正弦函数、余弦函数和正切函数。'), HumanMessage(content='可以用小学三年级小朋友能听懂的话解释吗？'), AIMessage(content='三角函数就是一种特殊的数学工具，可以帮助我们理解角度与三角形之间的关系，就像解开一个有趣的数学谜题一样。')])}

In [10]:
# 如果切换了 session_id 情况就会不同
with_message_history.invoke(
    {"ability": "数学", "input": "可以用小学三年级小朋友能听懂的话解释吗？"},
    config={"configurable": {"session_id": "abc456"}},
)

AIMessage(content='当我们在数学里谈到"20以内"，就是指所有小于等于20的数字哦！比如1、2、3、4、5、6、7、8、9、10、11、12、13、14、15、16、17、18、19和20。')

## 自定义对话历史的键值

In [82]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="用户ID",
            description="用户唯一标识",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="对话ID",
            description="对话唯一标识",
            default="",
            is_shared=True,
        ),
    ],
)

In [83]:
# with_message_history 此时的配置项是 user_id 和 conversation_id
with_message_history.config_specs

[ConfigurableFieldSpec(id='conversation_id', annotation=<class 'str'>, name='对话ID', description='对话唯一标识', default='', is_shared=True, dependencies=None),
 ConfigurableFieldSpec(id='user_id', annotation=<class 'str'>, name='用户ID', description='用户唯一标识', default='', is_shared=True, dependencies=None)]

In [84]:
# 此时调用就要提供 user_id 和 conversation_id
with_message_history.invoke(
    {"ability": "math", "input": "你好"},
    config={"configurable": {"user_id": "user-123", "conversation_id": "conv-1"}},
)

AIMessage(content='你好，请问有什么数学问题我可以帮助您解决呢？')

In [85]:
# 查看对话历史记录，键值已经变为由 user_id 和 conversation_id 的值构成的元组
store

{('user-123',
  'conv-1'): ChatMessageHistory(messages=[HumanMessage(content='你好'), AIMessage(content='你好，请问有什么数学问题我可以帮助您解决呢？')])}

In [89]:
# 实际上 with_message_history 对象也已经通过 _merge_configs 函数动态绑定了对话历史
with_message_history._merge_configs({
    "configurable": {
        "user_id": "user-123", 
        "conversation_id": "conv-1"
    }
})
# 下面的输出结果在每次调用 invoke 时会作为 config 参数被携带

{'configurable': {'user_id': 'user-123',
  'conversation_id': 'conv-1',
  'message_history': ChatMessageHistory(messages=[HumanMessage(content='你好'), AIMessage(content='你好，请问有什么数学问题我可以帮助您解决呢？')])}}

## 下面的例子使用了 output_messages_key

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOpenAI()})

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

with_message_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

# 自定义记忆管理

```python
class BaseChatMessageHistory(ABC):
    messages: List[BaseMessage]
    """必须重载：读取消息"""

    @abstractmethod
    def add_message(self, message: BaseMessage) -> None:
        """必须重载：增加消息历史"""
        raise NotImplementedError()

    @abstractmethod
    def clear(self) -> None:
        """必须重载：清除所有消息"""

```

# 基于内存的记忆管理实现

```python
from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.pydantic_v1 import BaseModel, Field


class ChatMessageHistory(BaseChatMessageHistory, BaseModel):
    """基于内存的消息历史管理，直接存储在内存列表中"""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_message(self, message: BaseMessage) -> None:
        """增加新消息"""
        self.messages.append(message)

    def clear(self) -> None:
        self.messages = []
```

# 基于文件的记忆管理实现

这里是更完整的例子：[https://github.com/langchain-ai/langserve/blob/main/examples/chat_with_persistence_and_user/server.py]

```python
import json
import logging
from pathlib import Path
from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import (
    BaseMessage,
    messages_from_dict,
    messages_to_dict,
)

logger = logging.getLogger(__name__)


class FileChatMessageHistory(BaseChatMessageHistory):
    """基于文件管理消息历史的示范，所有消息存储在本地JSON文件中

    参数:
        file_path: 保存JSON文件的路径
    """

    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        if not self.file_path.exists():
            self.file_path.touch()
            self.file_path.write_text(json.dumps([]))

    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        """从 JSON 文件提取历史消息"""
        items = json.loads(self.file_path.read_text())
        messages = messages_from_dict(items)
        return messages

    def add_message(self, message: BaseMessage) -> None:
        """将新消息增加到本地 JSON 文件"""
        messages = messages_to_dict(self.messages)
        messages.append(messages_to_dict([message])[0])
        self.file_path.write_text(json.dumps(messages))

    def clear(self) -> None:
        """清理所有消息历史"""
        self.file_path.write_text(json.dumps([]))

```